<a href="https://colab.research.google.com/github/NC25/gym_fishing/blob/master/spinning_up_td3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Steps for Custom algortihims

1.   Logger (probility distribution)

2.   Set seed

1.   Instantiate env

1.   Placeholders for computation

1.   Experience Buffer
2.  Helper Functions

7.   Running main loop












In [ ]:
%tensorflow_version 1.x
!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install stable-baselines[mpi]==2.10.0

TensorFlow 1.x selected.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-3).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
xvfb is already the newest version (2:1.19.6-1ubuntu4.4).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
!git clone https://github.com/boettiger-lab/gym_fishing.git
!python gym_fishing/setup.py sdist bdist_wheel
!pip install -e ./gym_fishing/
!ls
!cd gym_fishing

fatal: destination path 'gym_fishing' already exists and is not an empty directory.
running sdist
running egg_info
writing gym_fishing.egg-info/PKG-INFO
writing dependency_links to gym_fishing.egg-info/dependency_links.txt
writing top-level names to gym_fishing.egg-info/top_level.txt
reading manifest file 'gym_fishing.egg-info/SOURCES.txt'
writing manifest file 'gym_fishing.egg-info/SOURCES.txt'

running check


creating gym_fishing-0.0.2
creating gym_fishing-0.0.2/gym_fishing.egg-info
copying files to gym_fishing-0.0.2...
copying gym_fishing.egg-info/PKG-INFO -> gym_fishing-0.0.2/gym_fishing.egg-info
copying gym_fishing.egg-info/SOURCES.txt -> gym_fishing-0.0.2/gym_fishing.egg-info
copying gym_fishing.egg-info/dependency_links.txt -> gym_fishing-0.0.2/gym_fishing.egg-info
copying gym_fishing.egg-info/top_level.txt -> gym_fishing-0.0.2/gym_fishing.egg-info
Writing gym_fishing-0.0.2/setup.cfg
Creating tar archive
removing 'gym_fishing-0.0.2' (and everything under it)
running bdist_wheel

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
import tensorflow as tf
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)
import logging
tf.get_logger().setLevel(logging.ERROR)

In [ ]:
%matplotlib inline

import gym
import matplotlib
import numpy as np
import pandas as pd
import sys
import tensorflow as tf
from collections import defaultdict


matplotlib.style.use('ggplot')
import matplotlib.pyplot as plt

from stable_baselines.common.policies import LstmPolicy
from stable_baselines import TD3
from stable_baselines.td3.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.ddpg.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

In [ ]:
import gym_fishing

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


# Stable Baselines: Custom Actor-Critic Policy

In [ ]:
from stable_baselines.common.policies import ActorCriticPolicy, register_policy, nature_cnn
from stable_baselines.common.distributions import DiagGaussianProbabilityDistribution

"""
    Policy object that implements actor critic

    :sess: Current TensorFlow session
    :param ob_space: The observation space of env
    :param ac_space: (Gym Space) The action space of the environment
    :param n_env: (int) The number of environments to run for multiprocessing
    :param n_steps: (int) The number of steps to run for each environment
    :param n_batch: (int) The number of batchs to run (n_envs * n_steps)
    :param reuse: (bool) If the policy is reusable or not
    :param scale: (bool) whether or not to scale the input
    """

#Custom MLP Policy: Actor has 3 layers of size 128, Critic has 2 layers of size 32
# with a nature_cnn feature extractor

class CustomPolicy(ActorCriticPolicy):
    def __init__(self, sess, ob_space, ac_space, n_env, n_steps, n_batch, reuse=False, **kwargs):
        super(CustomPolicy, self).__init__(sess, ob_space, ac_space, n_env=n_env, n_steps=n_steps, n_batch=n_batch, reuse=reuse, scale=True)

        self._probability_distribution = DiagGaussianProbabilityDistribution
        self._pdtype = make_proba_dist_type(ac_space)
        self._policy = None
        self._proba_distribution = None
        self._value_fn = None
        self._action = None
        self._deterministic_action = None
        self.n_env = 2
        self.n_steps = 1000
        n_batch = 2000

        with tf.variable_scope("model", reuse=reuse):
            activation = tf.nn.relu

            output = nature_cnn(self.processed_obs, **kwargs)
            output = tf.layers.flatten(output)

            pi_h = output
            for i, layer_size in enumerate([128, 128, 128]):
                pi_h = activ(tf.layers.dense(pi_h, layer_size, name='pi_fc' + str(i)))
            pi_latent = pi_h

            vf_h = output
            for i, layer_size in enumerate([32, 32]):
                vf_h = activ(tf.layers.dense(vf_h, layer_size, name='vf_fc' + str(i)))
            value_fn = tf.layers.dense(vf_h, 1, name='vf')
            vf_latent = vf_h

            self._proba_distribution, self._policy, self.q_value = \
                self.pdtype.proba_distribution_from_latent(pi_latent, vf_latent, init_scale=0.01)

        self._value_fn = value_fn
        self._setup_init()

    def step(self, obs, state=None, mask=None, deterministic=False):
        if deterministic:
            action, value, neglogp = self.sess.run([self.deterministic_action, self.value_flat, self.neglogp],
                                                   {self.obs_ph: obs})
        else:
            action, value, neglogp = self.sess.run([self.action, self.value_flat, self.neglogp],
                                                   {self.obs_ph: obs})
        return action, value, self.initial_state, neglogp

    def proba_step(self, obs, state=None, mask=None):
        return self.sess.run(self.policy_proba, {self.obs_ph: obs})

    def value(self, obs, state=None, mask=None):
        return self.sess.run(self.value_flat, {self.obs_ph: obs})

  

In [ ]:
env = DummyVecEnv([lambda: gym.make('fishing-v1')])

n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

model = TD3(CustomPolicy, env, action_noise=action_noise, verbose=1)
# Train the agent
model.learn(total_timesteps=100000)

TypeError: ignored

# Spinning AI: Custom Neural Network and Policy

In [ ]:
def rnn():
  inputs = keras.Input(shape=(env.observation_space.shape))
  tf.keras.utils.normalize(inputs, axis=-1, order=2)
  layer_1 = layers.Dense(64, activation="relu")
  x = layer_1(inputs)
  layer_2 = layers.Dense(32, activation="relu")(x)
  x = layer_2(inputs)
  output = layers.Dense(8, activation='relu') 
  x = output(inputs)
  model = keras.Model(inputs=inputs, outputs=outputs, name=None)
  inputs.reshape(3, 2)
  model.add(layers.LSTM(64))
  model.add(layers.Dense(8))
  output = model.compile(loss=mean_squared_error, optimizer='adam')
  return output

rnn
#This will already be done when we make the policy or call agent
#model.compile(loss='mean_squared_error', optimizer='adam')



In [ ]:
def get_vars(scope):
  return[x for x in tf.global_variables() if scope in x.name]

def count_vars(scope):
  v = get_vars(scope)
  return sum(np.prod(var.shape.as_list()))


In [ ]:
!git clone https://github.com/openai/spinningup.git
!cd spinningup
!pip install -e ./spinningup/

In [ ]:
import spinningup
from logx import EpochLogger


In [ ]:
class ReplayBuffer:

  def __init__(self, obs_dim, act_dim, size):

    self.obs1_buf = np.zeros([size, obs_dim], dtype=np.float32)
    self.obs2_buf = np.zeros([size, obs_dim], dtype=np.float32)
    self.acts_buf = np.zeros([size, act_dim], dtype=np.float32)
    self.rewards_buf = np.zeros(size, dtype=np.float32)
    self.output_buf = np.zeros(sizem dtype=np.float32)
    self.ptr, self.size, self.max_size= 0,0, size

    def store(self, obs, act, rew, next_obs, done):
        self.obs1_buf[self.ptr] = obs
        self.obs2_buf[self.ptr] = next_obs
        self.acts_buf[self.ptr] = act
        self.rewards_buf[self.ptr] = rew
        self.outputs_buf[self.ptr] = done
        self.ptr = (self.ptr+1) % self.max_size
        self.size = min(self.size+1, self.max_size)

    def sample_batch(self, batch_size=32):
        idxs = np.random.randint(0, self.size, size=batch_size)
        return dict(obs1=self.obs1_buf[idxs],
                    obs2=self.obs2_buf[idxs],
                    acts=self.acts_buf[idxs],
                    rews=self.rewards_buf[idxs],
                    done=self.output_buf[idxs])




In [ ]:
import core

In [ ]:
def td3(env_fn, actor_critic=lstm_actor_critic(), ac_kwargs=dict(), seed=0, 
        steps_per_epoch=1000, epochs=300, replay_size=int(1e3), gamma=0.99, 
        polyak=0.995, pi_lr=1e-3, q_lr=1e-3, batch_size=100, start_steps=10000, 
        update_after=1000, update_every=50, act_noise=0.1, target_noise=0.2, 
        noise_clip=0.5, policy_delay=2, num_test_episodes=10, max_ep_len=1000, 
        logger_kwargs=dict(), save_freq=1):
  
  #polyak: averaging for target networks
  #pi_lr: learning rate for policy
  #q_lr: learneing rate for Q-networks
  #update_every: updates after every number of steps

  logger = EpochLogger(**logger_kwargs)
  logger.save_config(locals())

  tf.set_random_seed(seed)
  np.random.seed(seed)

#set dimensions
  env = env_fn()
  observation_dim = env.observation_space.shape[0]
  action_dim = env.action_space.shape[0]

  ac_kwargs['action_space'] = env.action_space
  #make all the dimensions the same bound


  #Target policy networks
  with tf.variable_scope('target'):
        pi_targ, _, _, _  = actor_critic(x2_ph, a_ph, **ac_kwargs)
    
  # Target Q networks
  with tf.variable_scope('target', reuse=True):

    #add noise
    epsilon = tf.random_normal(tf.shape(pi_targ), stddev=target_noise)
    epsilon = tf.clip_by_value(epsilon, -noise_clip, noise_clip)
    smoothed = pi_targ + epsilon
    smoothed = tf.clip_by_value(smoothed, -act_limit, act_limit)

    #set target Q values
    _, q1_targ, q2_targ, _ = actor_critic(x2_ph, a2, **ac_kwargs)

  #Experience Buffer
  replay_buffer = ReplayBuffer(observation_dim=observation_dim, action_dim=action_dim, size=replay_size)

  #Add Bellman Backup for stability
  min_q_target = tf.minimum(q1_target, q2_target)
  
  backup = tf.stop_gradient(r_ph + gamma*(1-d_p)*min_q_target)

  #Loss function for TD3
  pi_loss = -tf.reduce_mean(q1_pi)
  q1_loss = tf.reduce_mean((q1-backup)**2)
  q2_loss = tf.reduce.mean((q2-backup)**2)
  q_loss = q1_loss + q2_loss


  #optimizer
  pi_optimizer = tf.train.AdamOptimizer(learning_rate=pi_lr)
  q_optimizer = tf.train.AdamOptimizer(learning_rate=q_lr)
  train_pi_op = pi_optimizer.minimize(pi_loss, var_list=get_vars('main/pi'))
  train_q_op = q_optimizer.minimize(q_loss, var_list=get_vars('main/q'))

  sess = tf.Session()
  sess.run(tf.global_variables_initializer())
  sess.run(target_init)

  #save weights
  logger.setup_tf_saver(sess, inputs={'x': x_ph, 'a': a_ph}, outputs={'pi': pi, 'q1': q1, 'q2': q2})

  def step(observation, noise_scale):
  action = sess.run(pi, feed_dict={x_ph: observation.reshape(1, -1)})
  action += noise_scale + np.random.randn(action_dim)
  return np.clip(a, -act_limit, act_limit)

  def sample():
  for i in range(num_test_episodes):
    observation = env.reset()
    while ep_len <= max_ep_len:
      observation, reward, done, _ = env.step(action(observation, 0))
      ep_return += reward
      ep_len += 1
      d = False if ep_len==max_ep_len else d
      replay_buffer.store(observation, action, reward, observation_2, done)
      observation = observation_2
    logger.store(TestEpRet=ep_ret, TestEpLen=ep_len)



In [ ]:
from spinup.utils.run_utils import setup_logger_kwargs
logger_kwargs = setup_logger_kwargs(args.exp_name, args.seed)


td3()